# Installing Istio on a Minikube Cluster
- reference link: https://kubebyexample.com/learning-paths/istio/install
- learning points:
    - Start the minikube instance, and verify that MetalLB is installed
    - Install Istio on the minikube cluster
    - Get the Istio ingress service endpoint
    - Deploy the Istio add-ons
    - Create an example application
    - Generate traffic for the example application
    - Access the Istio dashboard


### Start the minikube instance, and verify that MetalLB is installed
- start minikube cluster with minikube start command, specified cpus, memory, profile and driver
    - minikube start --cpus=4 --memory=8g --driver=virtualbox -p minikube2
- verify metallb pods is running with kubectl get pods with specified namespace
    - kubectl get pods -n metallb-system
- write metallb loadbalancer config to yaml
    - kubectl get configmap config -n metallb-system -o yaml

### Install Istio on the minikube cluster
- set the istio version we need to get, get newer version than in the post
    - export ISTIO_VERSION=1.17.3
    - release reference link: https://istio.io/latest/news/releases/
- download install file with wget command
    - wget -c -nv \
        https://github.com/istio/istio/releases/download/${ISTIO_VERSION}/istio-${ISTIO_VERSION}-linux-amd64.tar.gz
- extract file with tar command, remove file with rm command
    - tar -xzf istio-${ISTIO_VERSION}-linux-amd64.tar.gz
    - rm -v istio-${ISTIO_VERSION}-linux-amd64.tar.gz
- update PATH environ variable which include user home bin directory
    - export PATH="${PATH}:${HOME}/bin"
- create ~/bin directory if not exist
    - mkdir -vp ~/bin
- move and add execute permission to bin directory
    - install --mode 0755 istio-${ISTIO_VERSION}/bin/istioctl ~/bin/
- append new path environment to ~/.bashrc
    - echo 'export PATH="${PATH}:${HOME}/bin"' | tee -a ~/.bashrc
        export PATH="${PATH}:${HOME}/bin"
- check command link directory with which command
    - which istioctl
- use istioctl command line to check minikube cluster
    - istioctl experimental precheck
- install istio to minikube cluster
    - istioctl install --set profile=demo -y
- list all deployments and pods
    - kubectl get deployments,pods -n istio-system
    - we have ingressgateway and egressgateway deployments, pods

### Get the Istio ingress service endpoint.
- get services
    - kubectl get services -n istio-system
    - notes
        - ingressgateway is a loadbalancer which have external ip provided by metallb
- get external ip address of istio-ingressgateway
    - istio-ingressgateway is a loadblancer service
    - export INGRESS_HOST=$(kubectl get service \
        istio-ingressgateway -n istio-system \
        -o jsonpath='{.status.loadBalancer.ingress[0].ip}')
- get istio-ingressgateway port for http request
    - export INGRESS_PORT=$(kubectl get service \
        istio-ingressgateway -n istio-system \
        -o jsonpath='{.spec.ports[?(@.name=="http2")].port}')
- get istio-ingressgateway port for https request
    export SECURE_INGRESS_PORT=$(kubectl get service \
        istio-ingressgateway -n istio-system \
        -o jsonpath='{.spec.ports[?(@.name=="https")].port}')
- get ingressgateway url path
    - export GATEWAY_URL="${INGRESS_HOST}:${INGRESS_PORT}"

### Deploy the Istio add-ons.
- list istio add one
    - ls istio-${ISTIO_VERSION}/samples/addons/
- apply add on to k8s cluster
    - kubectl apply -f istio-${ISTIO_VERSION}/samples/addons
- check add-on deployments status
    - kubectl get deployments -n istio-system



### Create an example application
- create k8s example application
    - kubectl apply -f \
        istio-${ISTIO_VERSION}/samples/helloworld/helloworld.yaml
- verify deployment and pods
    - kubectl get deployments,pods -l app=helloworld
    - notes
        - we have two deployment and each deployment have two pod
- verify the serivce have to external ip
    - kubectl get services -l app=helloworld
- create gateway and virtual service 
    - kubectl apply -f \
        istio-${ISTIO_VERSION}/samples/helloworld/helloworld-gateway.yaml
    - notes:
        - we need to know on the concept level what is the purpose of gateway and virtual service
- get url path from virtual service
    - kubectl get virtualservice helloworld \
        -o jsonpath='{.spec.http[0].match[0].uri}{"\n"}'
    - notes
        - virtual service will define url routing

### Generate traffic for the example application
- make request to url path define in virtual service
    - curl -vk# "http://${GATEWAY_URL}/hello"
- Inspect the commands contained in the loadgen.sh script
    - grep -v '^#' \
        istio-${ISTIO_VERSION}/samples/helloworld/loadgen.sh
    - loadgen.sh script generate request using while loop and curl command
- run loadgen.sh script
    - ./istio-${ISTIO_VERSION}/samples/helloworld/loadgen.sh

### Access the Istio dashboard
- run istio dashboard
    - istioctl dashboard kiali
- in this section we just explore front-end kiali
    - istio-gateway is where your request reach
    - gateway forward request virtual service account which act as an balancer to forward request to deployment 

### Important information you get from istio
- INGRESS_HOST: external ip of ingress service
- INGRESS_PORT: port on service which will accept http request
- SECURE_INGRESS_PORT: service port which accept https request
- GATEWAY_URL: combination of ingress_host and ingress port
kubectl label namespace default istio-injection=enabled --overwrite
 

### Note about istio on k8s
- istio is just an application on k8s with all the concept of deployment and pods
- to use istio for ingress control on k8s we just need to care about istio-ingressgateway
- to use istio for ingress control on k8s get:
    - ingress service host
    - ingress port for http
    - ingress port for https
- application can be on the same cluster or on different cluster
- kiali is front-end tools for ingress control management
    - remember to select namespace which application is live one
- config ingress to application:
    - we need running application (Deployment or StatefulSet)
    - we need to config Gateway and Virtual Service account
- analyze helloworld example
    - deployment
        - two deployments. Each has container's running port 5000
    - gateway:
        - define load balancer
        - selector: which pods to process request
        - server: port, request type, host that the gateway or loadbalancer will handle
        - gateway api reference link: https://istio.io/latest/docs/reference/config/networking/gateway/
    - virtual service
        - define routing rule
        - gatewate will be input for virtual service
        - virtual service api reference link: https://istio.io/latest/docs/reference/config/networking/virtual-service/
        - in hello world virtual service will forward request to helloworld service using service name
        
- gatewayurl is where istio service expose to request outsite k8s cluster